## Imports and Functions

We begin by loading the necessary packages/modules.

In [1]:
import os
import pandas as pd
import numpy as np

## Calculating Summary Statistics

We calculate and display summary statistics of our parameter estimation results as shown in [Juri, 2014](<..\references\Juri, Alessandro (2014) Lending Values and Liquidity Risk.txt>).

In [2]:
df = pd.read_csv(os.path.join('..', 'models', 'liq_param', 'liq_estimates.csv'))

stats = df.groupby('Ticker').apply(
    lambda group: pd.Series({
        'Mean Gamma Estimate': group[group['Gamma Estimate'] > 0]['Gamma Estimate'].mean(),
        'Median CI Upper Bound': group[group['Gamma Estimate'] > 0]['CI Upper Bound (Gamma)'].median(),
        'Median CI Lower Bound': group[group['Gamma Estimate'] > 0]['CI Lower Bound (Gamma)'].median(),
        '25th Percentile Gamma Estimate': group[group['Gamma Estimate'] > 0]['Gamma Estimate' ].quantile(0.25),
        '50th Percentile Gamma Estimate': group[group['Gamma Estimate'] > 0]['Gamma Estimate'].quantile(0.50),
        '75th Percentile Gamma Estimate': group[group['Gamma Estimate'] > 0]['Gamma Estimate'].quantile(0.75),
        'Positive Gamma Days (%)': (group['Gamma Estimate'] > 0).mean() * 100,
        'Full Model Significant Days (%)': group['Full Model Significance'].mean() * 100,
        'Gamma Significant Days (%)': group['Gamma Significance'].mean() * 100,
        'Positive Gamma Significant Days (%)': group[group['Gamma Significance']]['Gamma Estimate'].gt(0).mean() * 100,
        'Negative Gamma Significant Days (%)': group[group['Gamma Significance']]['Gamma Estimate'].lt(0).mean() * 100
    })
)

# reset index if needed and handle NaN cases
stats.reset_index(inplace=True)
stats.fillna(0, inplace=True)

display(stats)
stats.to_csv(os.path.join('..', 'models', 'liq_param', 'liq_summary.csv'), index=False)

,Ticker,Mean Gamma Estimate,Median CI Upper Bound,Median CI Lower Bound,25th Percentile Gamma Estimate,50th Percentile Gamma Estimate,75th Percentile Gamma Estimate,Positive Gamma Days (%),Full Model Significant Days (%),Gamma Significant Days (%),Positive Gamma Significant Days (%),Negative Gamma Significant Days (%)
0,ABBN,6.384515e-11,7.918468e-11,2.705913e-11,3.081009e-11,5.796509e-11,7.698502e-11,100.000000,6.779661,89.830508,100.0,0.0
1,CLN,7.586625e-10,6.611957e-10,7.148434e-11,2.139071e-10,3.413054e-10,6.245205e-10,100.000000,1.694915,66.101695,100.0,0.0
2,DOKA,1.675057e-07,1.492747e-07,9.966724e-09,5.121987e-08,7.251144e-08,1.400953e-07,100.000000,5.084746,61.016949,100.0,0.0
3,GIVN,1.786180e-08,2.128580e-08,3.019803e-09,5.722100e-09,1.181413e-08,2.237157e-08,100.000000,5.660377,73.584906,100.0,0.0
4,KUD,4.370480e-06,5.927785e-07,-2.087197e-07,2.971381e-08,1.200926e-07,4.947797e-07,83.050847,8.474576,20.338983,100.0,0.0
5,LISN,1.182801e-05,1.241056e-05,1.421683e-06,2.941765e-06,7.729151e-06,1.282910e-05,100.000000,0.000000,71.186441,100.0,0.0
6,LONN,1.553094e-09,1.201127e-09,3.635167e-10,4.932418e-10,7.591466e-10,1.308801e-09,100.000000,5.454545,85.454545,100.0,0.0
7,NESN,3.362634e-11,3.528533e-11,1.390054e-11,1.444125e-11,2.499049e-11,3.820972e-11,98.181818,9.090909,94.545455,100.0,0.0
8,SCHN,3.166630e-08,2.976613e-08,6.047592e-09,9.820157e-09,1.587605e-08,3.013803e-08,100.000000,5.454545,83.636364,100.0,0.0
9,SCMN,3.673872e-09,3.580228e-09,1.298832e-09,1.346055e-09,2.217755e-09,3.503798e-09,100.000000,3.448276,87.931034,100.0,0.0


## Calculating Moving Averages

We calculate the 30-day moving average and average over the entire sample period (21.12.2023 to 18.03.2024) and save it as [`gamma_table.csv`](<..\models\liq_param\gamma_table.csv>) to use for visualization.

In [3]:
df = pd.read_csv(os.path.join('..', 'models', 'liq_param', 'liq_estimates.csv'))

# check data types
df['Date'] = pd.to_datetime(df['Date'])
df['Ticker'] = df['Ticker'].astype(str)

# set 'Date' as index
df.set_index('Date', inplace=True)

# group by 'Ticker' and calc avg
df['30d_MA_Gamma'] = df.groupby('Ticker')['Gamma Estimate'].rolling(window=30).mean().reset_index(0, drop=True)
df['Expanding_Avg_Gamma'] = df.groupby('Ticker')['Gamma Estimate'].expanding().mean().reset_index(0, drop=True)

# calc static avg
ticker_avg = df.groupby('Ticker')['Gamma Estimate'].mean()

# map static avg back to the main
df['Gamma_Mean'] = df['Ticker'].map(ticker_avg)

# reset index for pivot
df.reset_index(inplace=True)

# pivot
pivot_30d = df.pivot(index='Date', columns='Ticker', values='30d_MA_Gamma')
pivot_cum_avg = df.pivot(index='Date', columns='Ticker', values='Expanding_Avg_Gamma')
pivot_avg = df.pivot(index='Date', columns='Ticker', values='Gamma_Mean')
daily_gamma = df.pivot(index='Date', columns='Ticker', values='Gamma Estimate')

# merge
merged_df = pd.concat([daily_gamma.add_suffix('_Daily'), pivot_30d.add_suffix('_30d_MA'), pivot_cum_avg.add_suffix('_Expanding_Avg'), pivot_avg.add_suffix('_Avg')], axis=1)
merged_df.reset_index(inplace=True)

# display and save
display(merged_df)
merged_df.to_csv(os.path.join('..', 'models', 'liq_param', 'gamma_table.csv'))

Ticker,Date,ABBN_Daily,CLN_Daily,DOKA_Daily,GIVN_Daily,KUD_Daily,LISN_Daily,LONN_Daily,NESN_Daily,SCHN_Daily,...,LISN_Avg,LONN_Avg,NESN_Avg,SCHN_Avg,SCMN_Avg,SIKA_Avg,SRENH_Avg,UBSG_Avg,UHR_Avg,VLRT_Avg
0,2023-12-21,1.021534e-10,4.062741e-10,5.072722e-08,1.744112e-08,4.670459e-07,4.785544e-06,1.578303e-09,3.089278e-11,1.983773e-08,...,0.000012,1.553094e-09,3.291485e-11,3.166630e-08,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,0.009232
1,2023-12-22,7.511271e-11,7.127582e-10,6.053584e-08,1.695420e-08,3.049259e-08,2.671529e-06,2.159203e-09,5.528292e-11,1.180879e-07,...,0.000012,1.553094e-09,3.291485e-11,3.166630e-08,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,0.009232
2,2023-12-27,7.792762e-11,3.451134e-10,9.394500e-08,1.578136e-08,2.240249e-08,9.262422e-06,8.191345e-10,4.434086e-11,1.476918e-08,...,0.000012,1.553094e-09,3.291485e-11,3.166630e-08,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,NaN
3,2023-12-28,6.964727e-11,2.109479e-09,1.291641e-07,3.405246e-08,-2.433812e-09,5.075264e-05,7.636614e-10,2.705036e-11,1.587605e-08,...,0.000012,1.553094e-09,3.291485e-11,3.166630e-08,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,0.009232
4,2023-12-29,1.468496e-10,3.597517e-10,8.798339e-08,1.569078e-08,2.971381e-08,2.404844e-05,6.900932e-10,1.911861e-11,NaN,...,0.000012,1.553094e-09,3.291485e-11,NaN,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,0.009232
5,2024-01-03,5.207383e-11,2.251327e-10,6.652343e-08,1.251231e-08,7.917554e-08,5.916723e-05,1.143931e-09,3.542762e-11,NaN,...,0.000012,1.553094e-09,3.291485e-11,NaN,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,NaN
6,2024-01-04,1.494187e-10,3.845500e-10,9.441561e-08,8.046845e-08,3.190080e-07,1.238532e-05,2.673197e-08,2.778678e-11,NaN,...,0.000012,1.553094e-09,3.291485e-11,NaN,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,NaN,NaN
7,2024-01-05,6.237246e-11,2.320982e-10,3.890811e-08,9.759413e-09,1.094838e-06,1.699475e-05,6.028246e-10,8.207922e-11,NaN,...,0.000012,1.553094e-09,3.291485e-11,NaN,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,NaN
8,2024-01-08,1.497857e-10,2.114418e-10,5.628584e-08,3.667830e-08,5.182018e-06,8.525498e-06,1.170121e-09,1.300716e-10,2.300036e-09,...,0.000012,1.553094e-09,3.291485e-11,3.166630e-08,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,NaN
9,2024-01-09,1.798929e-10,2.740322e-10,1.567176e-07,5.442764e-09,2.351111e-08,2.404168e-05,5.742930e-10,3.038430e-11,2.524244e-08,...,0.000012,1.553094e-09,3.291485e-11,3.166630e-08,3.673872e-09,6.916961e-10,2.644731e-10,3.745429e-11,1.469624e-09,0.009232
